In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
pretrain_features = {
    "learning_rate": 0.01,
    "batch_size": 256,
    "epochs": 1000,
    "eval_size": 4*256,
    "momentum": 0.005,
    "weight_decay": 0.0001,
}

In [20]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None
    
    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    scaler = StandardScaler()
    x_pretrain = scaler.fit_transform(x_pretrain)
    x_train = scaler.transform(x_train)
    x_test_transed = scaler.transform(x_test)
    x_test[x_test.columns] = x_test_transed

    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [21]:
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()

c:\Users\linan\.conda\envs\DL\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [22]:
class AE(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1000, 1000),
            nn.BatchNorm1d(1000),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(1000, 1000)
            )

        self.decoder = nn.Sequential(
            nn.Linear(1000, 1000),
            nn.BatchNorm1d(1000),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(1000, 1000)
            )
            
        for m in self.modules():
            if isinstance(m, nn.Linear):    
                nn.init.xavier_uniform_(m.weight)
        
        
    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def encode(self, x):
        return self.encoder(x)

In [23]:
eval_size = pretrain_features["eval_size"]
batch_size = pretrain_features["batch_size"]
ae_model = AE()
ae_model.train()
ae_model.to(device)

def train_autoencoder():
    x_tr, x_val, y_tr, y_val = train_test_split(x_pretrain, y_pretrain, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.Adam(ae_model.parameters(), lr=pretrain_features['learning_rate'], weight_decay=pretrain_features['weight_decay'])
    # optimizer = torch.optim.SGD(ae_model.parameters(), lr=pretrain_features['learning_rate'], momentum=pretrain_features['momentum'], weight_decay=pretrain_features['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = pretrain_features['epochs']
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, _] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, _] in val_loader:
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        
        if(epoch % 10 == 0):
            print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

train_autoencoder()

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: train loss: 1.0943057450232494, val loss: 0.9604995846748352
Epoch 11: train loss: 0.7546058118012324, val loss: 0.8425064980983734
Epoch 00014: reducing learning rate of group 0 to 3.0000e-03.
Epoch 21: train loss: 0.6990577956306822, val loss: 0.7889628261327744
Epoch 31: train loss: 0.687005219779658, val loss: 0.7803559750318527
Epoch 00036: reducing learning rate of group 0 to 9.0000e-04.
Epoch 41: train loss: 0.6488821160477853, val loss: 0.7431006133556366
Epoch 00046: reducing learning rate of group 0 to 2.7000e-04.
Epoch 51: train loss: 0.6228825188041396, val loss: 0.7169996052980423
Epoch 00061: reducing learning rate of group 0 to 8.1000e-05.
Epoch 61: train loss: 0.6196563172402705, val loss: 0.7176687717437744
Epoch 71: train loss: 0.6101881380916148, val loss: 0.7063692212104797
Epoch 00075: reducing learning rate of group 0 to 2.4300e-05.
Epoch 81: train loss: 0.6071158064147765, val loss: 0.704039916396141
Epoch 00090: reducing learning rate of group 0 to 7.29

In [24]:
def get_regression_model(X, y):

    class Model(nn.Module):
            """
            The model class, which defines our feature extractor used in pretraining.
            """
            def __init__(self):
                """
                The constructor of the model.
                """
                super().__init__()
                # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
                # and then used to extract features from the training and test data.
                self.seq = nn.Sequential(
                    nn.Linear(1000, 40),
                    nn.BatchNorm1d(40),
                    nn.LeakyReLU(0.01),
                    nn.Dropout(0.6),
                    nn.Linear(40, 1)
                )

                for m in self.modules():
                    if isinstance(m, nn.Linear):    
                        nn.init.xavier_uniform_(m.weight)

            def forward(self, x):
                """
                The forward pass of the model.

                input: x: torch.Tensor, the input to the model

                output: x: torch.Tensor, the output of the model
                """
                # TODO: Implement the forward pass of the model, in accordance with the architecture 
                # defined in the constructor.
                x = self.seq(x)
                return x
    
    """
    This function returns the regression model used in the pipeline.

    input: None

    output: model: sklearn compatible model, the regression model
    """
    # TODO: Implement the regression model. It should be able to be trained on the features extracted
    # by the feature extractor.
    model = Model()
    model.to(device)
    model.train()

    x_tr, x_val, y_tr, y_val = train_test_split(X, y, test_size=10, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
    

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
    # optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.4, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 10000
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, y] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
            loss_tr = loss.item()
        for [x, y] in val_loader:
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss_val = loss.item()
        scheduler.step(loss_val)
        if(epoch % 10 == 0):
            print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-8):
            print(f"Early stop at epoch {epoch+1}, train loss: {loss_tr}, val loss: {loss_val}")
            break
    return model


featured_x_pretrain = ae_model.encode(torch.tensor(x_pretrain, dtype=torch.float).to(device)).detach().cpu().numpy()
one_model = get_regression_model(featured_x_pretrain, y_pretrain)

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 1: train loss: 12.371824264526367, val loss: 9.59903621673584
Epoch 00017: reducing learning rate of group 0 to 3.0000e-03.
Epoch 00026: reducing learning rate of group 0 to 9.0000e-04.
Epoch 00032: reducing learning rate of group 0 to 2.7000e-04.
Epoch 00042: reducing learning rate of group 0 to 8.1000e-05.
Epoch 00048: reducing learning rate of group 0 to 2.4300e-05.
Epoch 00054: reducing learning rate of group 0 to 7.2900e-06.
Epoch 00060: reducing learning rate of group 0 to 2.1870e-06.
Epoch 00066: reducing learning rate of group 0 to 6.5610e-07.
Epoch 00072: reducing learning rate of group 0 to 1.9683e-07.
Epoch 00081: reducing learning rate of group 0 to 5.9049e-08.
Epoch 00087: reducing learning rate of group 0 to 1.7715e-08.
Epoch 00093: reducing learning rate of group 0 to 5.3144e-09.
Early stop at epoch 93


In [25]:
result_file = "results-henhuo.csv"

In [26]:
# import huberregression
from sklearn.linear_model import HuberRegressor

In [27]:
huber = HuberRegressor()
featured_x_train = one_model(ae_model.encoder(torch.tensor(x_train, dtype=torch.float).to(device))).squeeze(-1).detach().cpu().numpy()
huber.fit(featured_x_train.reshape(-1, 1), y_train)

y_pred = np.zeros(x_test.shape[0])
featured_x_test = one_model(ae_model.encoder(torch.tensor(x_test.to_numpy(), dtype=torch.float).to(device))).squeeze(-1).detach().cpu().numpy()
y_pred = huber.predict(featured_x_test.reshape(-1, 1))

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv(result_file, index_label="Id")
print(f"Predictions saved to {result_file}, all done!")

Predictions saved to results-henhuo.csv, all done!
